In [43]:
import polars as pl
import numpy as np
from tqdm import tqdm
from scipy.sparse import csr_matrix
import implicit
import pandas as pd
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
RANDOM_STATE = 42
N_PREDICTIONS = 100


In [2]:
import seaborn as sns
sns.set()

### Чтение и подготовка данных для анализа 

In [ ]:
df_names = pd.read_json('/Users/anastasiakucina/hahaha/names (3).jsonl')

In [4]:
df_names.head(10)

,clean_names,slash_names,parenthesis
0,Смотритель музейный,[],[]
1,Ведущий менеджер по работе с физическими лицами,[],[]
2,Бухгалтер,[],[по расчету зарплаты]
3,Пекарь,[],"[Токсово, Привокзальная, 16]"
4,Торговый представитель,[],[г. Абакан]
5,Менеджер по качеству,[],[]
6,Менеджер по продажам,[Sales manager],[]
7,Врач по ЛФК,[],[]
8,Монтажник СТиЖБК,[],[]
9,Подсобный рабочий,[],[строительный объект]


In [44]:
df_vacancies = pd.read_json('/Users/anastasiakucina/hahaha/vacancies_no_description.jsonl')

In [6]:
df_vacancies.head()

,vacancy_id,name,company.id,keySkills.keySkill,compensation.from,compensation.to,compensation.currencyCode,area.id,area.regionId,employment,workSchedule,workExperience
0,v_862116,Смотритель музейный,c_162972,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience
4,v_312507,Торговый представитель (г. Абакан),c_206699,"[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3


In [ ]:
df_vacancies['clean_name'] = df_names['clean_names']

In [ ]:
df_vacancies.rename(columns={"company.id": "company_id", "keySkills.keySkill": "keySkills", "compensation.from":"compensation_from", "compensation.to":"compensation_to", "compensation.currencyCode":"compensation_currencyCode", "area.id": "area_id",  "area.regionId" : 'area_regionId'}, inplace = True)

In [ ]:
df_vacancies['area_regionId_encoder'] = labelencoder.fit_transform(df_vacancies['area_regionId'].values)
df_vacancies['area_id_encoder'] = labelencoder.fit_transform(df_vacancies['area_id'].values)
df_vacancies['company_id_encoder'] = labelencoder.fit_transform(df_vacancies['company_id'].values)


In [ ]:
df_vacancies_ru = df_vacancies.query('compensation_currencyCode == "RUR"')
df_vacancies_ru.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2209572 entries, 0 to 2734128
Data columns (total 16 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   vacancy_id                 object 
 1   name                       object 
 2   company_id                 object 
 3   keySkills                  object 
 4   compensation_from          float64
 5   compensation_to            float64
 6   compensation_currencyCode  object 
 7   area_id                    object 
 8   area_regionId              object 
 9   employment                 object 
 10  workSchedule               object 
 11  workExperience             object 
 12  clean_name                 object 
 13  area_regionId_encoder      int64  
 14  area_id_encoder            int64  
 15  company_id_encoder         int64  
dtypes: float64(2), int64(3), object(11)
memory usage: 286.6+ MB


In [ ]:
## employment
df_vacancies_ru['employment'].unique()

array(['full', 'part', 'project', 'probation', 'volunteer'], dtype=object)

In [ ]:
def employment_code(row) :

    if row == 'full': 
        return 5
    elif row == 'part':
        return 4
    elif row == 'project':
        return 3
    elif row== 'probation':
        return 2
    else:
        return 1


In [ ]:
df_vacancies_ru['employment_encoder'] = df_vacancies_ru['employment'].apply(employment_code)
df_vacancies_ru.head()


/var/folders/gz/46_zg8xd6qqgn1sxylfq7g8m0000gn/T/ipykernel_3657/3453994751.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vacancies_ru['employment_encoder'] = df_vacancies_ru['employment'].apply(employment_code)


,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder
0,v_862116,Смотритель музейный,c_162972,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,81,5806,109013,5
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,52,4218,235992,5
4,v_312507,Торговый представитель (г. Абакан),c_206699,"[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5


In [ ]:
df_vacancies_ru['workSchedule'].unique()

array(['fullDay', 'flyInFlyOut', 'flexible', 'shift', 'remote'],
      dtype=object)

In [ ]:
def schedule_code(row) :
    if row == 'fullDay':
        return 5
    elif row == 'shift':
        return 4
    elif row == 'remote':
        return 3
    elif row == 'flexible':
        return 2
    else:
        return 1


In [ ]:
df_vacancies_ru['workSchedule_encoder'] = df_vacancies_ru['workSchedule'].apply(schedule_code)
df_vacancies_ru

/var/folders/gz/46_zg8xd6qqgn1sxylfq7g8m0000gn/T/ipykernel_3657/3743479919.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vacancies_ru['workSchedule_encoder'] = df_vacancies_ru['workSchedule'].apply(schedule_code)


,vacancy_id,name,company_id,keySkills,compensation_from,compensation_to,compensation_currencyCode,area_id,area_regionId,employment,workSchedule,workExperience,clean_name,area_regionId_encoder,area_id_encoder,company_id_encoder,employment_encoder,workSchedule_encoder
0,v_862116,Смотритель музейный,c_162972,"[Пользователь ПК, Работа в команде, Умение пла...",16500.0,NaN,RUR,a_4761,ar_33,full,fullDay,noExperience,Смотритель музейный,32,4181,69972,5,5
1,v_288642,Ведущий менеджер по работе с физическими лицами,c_208672,"[Активные продажи, Холодные продажи, Кредитные...",50000.0,NaN,RUR,a_744,ar_2,full,fullDay,noExperience,Ведущий менеджер по работе с физическими лицами,17,6731,120750,5,5
2,v_1840054,Бухгалтер (по расчету зарплаты),c_198109,None,50000.0,65000.0,RUR,a_6223,ar_78,full,fullDay,between3And6,Бухгалтер,81,5806,109013,5,5
3,v_2346232,"Пекарь (Токсово, Привокзальная, 16)",c_6137,None,38500.0,42000.0,RUR,a_4795,ar_51,full,fullDay,noExperience,Пекарь,52,4218,235992,5,5
4,v_312507,Торговый представитель (г. Абакан),c_206699,"[Продуктивность, Клиентоориентированность, Упр...",60000.0,NaN,RUR,a_6837,ar_4,full,fullDay,between1And3,Торговый представитель,39,6487,118557,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2734124,v_2484959,Сборщик-упаковщик,c_203256,"[Пользователь ПК, Умение работать в команде, У...",40000.0,90000.0,RUR,a_5387,ar_71,full,flexible,noExperience,Сборщик-упаковщик,74,4876,114733,5,2
2734125,v_205163,Сварщик на полуавтомат,c_158695,"[Желание работать и зарабатывать, Высокая энер...",80000.0,130000.0,RUR,a_5527,ar_69,full,fullDay,between1And3,Сварщик на полуавтомат,71,5032,65219,5,5
2734126,v_639897,Главный инженер / Технический директор,c_209365,"[Контроль исправности оборудования, Инженерные...",200000.0,NaN,RUR,a_1756,ar_41,full,fullDay,between1And3,Главный инженер,41,842,121520,5,5
2734127,v_1636531,"Провизор/Фармацевт (г.Адыгейск, 20 км от Красн...",c_246244,"[Предпечатная подготовка, Статистический анализ]",25000.0,NaN,RUR,a_3403,ar_60,full,fullDay,noExperience,Фармацевт,62,2673,162497,5,5


In [ ]:
df_vacancies_ru['workExperience'].unique()

array(['noExperience', 'between3And6', 'between1And3', 'moreThan6'],
      dtype=object)

In [ ]:
def experience_code(row) :
    if row == 'moreThan6':
        return 4
    elif row == 'between3And6':
        return 3
    elif row == 'between1And3':
        return  2
    else:
        return 1


In [ ]:
df_vacancies_ru['workExperience_encoder'] = df_vacancies_ru['workExperience'].apply(experience_code)

/var/folders/gz/46_zg8xd6qqgn1sxylfq7g8m0000gn/T/ipykernel_3657/2919616377.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vacancies_ru['workExperience_encoder'] = df_vacancies_ru['workExperience'].apply(experience_code)


### Корреляционный анализ 


In [20]:
numeric = ['employment_encoder', 'workSchedule_encoder', 'workExperience_encoder', 'compensation_from', 'compensation_to', 'company_id_encoder', 'area_id_encoder', 'area_regionId_encoder']

df_vacancies_ru[numeric].corr(method='spearman')

,employment_encoder,workSchedule_encoder,workExperience_encoder,compensation_from,compensation_to,company_id_encoder,area_id_encoder,area_regionId_encoder
employment_encoder,1.000000,0.241685,0.189495,0.115123,0.015340,0.031414,-0.000339,-0.001691
workSchedule_encoder,0.241685,1.000000,0.141493,-0.084158,-0.135072,0.006975,0.000174,0.012296
workExperience_encoder,0.189495,0.141493,1.000000,0.393191,0.315146,-0.008747,-0.054278,-0.024251
compensation_from,0.115123,-0.084158,0.393191,1.000000,0.869907,-0.010165,-0.112878,-0.033406
compensation_to,0.015340,-0.135072,0.315146,0.869907,1.000000,-0.002737,-0.116061,-0.034921
company_id_encoder,0.031414,0.006975,-0.008747,-0.010165,-0.002737,1.000000,-0.002599,0.001551
area_id_encoder,-0.000339,0.000174,-0.054278,-0.112878,-0.116061,-0.002599,1.000000,0.108563
area_regionId_encoder,-0.001691,0.012296,-0.024251,-0.033406,-0.034921,0.001551,0.108563,1.000000


In [21]:
df_vacancies_ru[numeric].corr(method='kendall')

,employment_encoder,workSchedule_encoder,workExperience_encoder,compensation_from,compensation_to,company_id_encoder,area_id_encoder,area_regionId_encoder
employment_encoder,1.000000,0.226783,0.180147,0.095179,0.012645,0.025577,-0.000284,-0.001411
workSchedule_encoder,0.226783,1.000000,0.127856,-0.060840,-0.102298,0.005095,0.000115,0.009769
workExperience_encoder,0.180147,0.127856,1.000000,0.317737,0.253291,-0.006809,-0.043192,-0.019374
compensation_from,0.095179,-0.060840,0.317737,1.000000,0.720144,-0.006814,-0.077521,-0.025189
compensation_to,0.012645,-0.102298,0.253291,0.720144,1.000000,-0.001750,-0.079754,-0.025707
company_id_encoder,0.025577,0.005095,-0.006809,-0.006814,-0.001750,1.000000,-0.001774,0.001047
area_id_encoder,-0.000284,0.000115,-0.043192,-0.077521,-0.079754,-0.001774,1.000000,0.086388
area_regionId_encoder,-0.001411,0.009769,-0.019374,-0.025189,-0.025707,0.001047,0.086388,1.000000


In [ ]:
df_vacancies_ru.to_csv(​ "/Users/anastasiakucina/hahaha/vacancies_ru.csv"​) #укажи путь куда сохранять 

## Обработка train_data



In [22]:
vacancies = set(df_vacancies.query('compensation_currencyCode != "RUR"')['vacancy_id'].to_list())

In [23]:
df_train = pd.read_json('/Users/anastasiakucina/hahaha/hh_recsys_train_hh.jsonl')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3463064 entries, 0 to 3463063
Data columns (total 5 columns):
 #   Column       Dtype 
---  ------       ----- 
 0   user_id      object
 1   session_id   object
 2   vacancy_id   object
 3   action_type  object
 4   action_dt    object
dtypes: object(5)
memory usage: 132.1+ MB


In [24]:
df_train_mini = df_train[:100]

In [25]:
df_train_mini

,user_id,session_id,vacancy_id,action_type,action_dt
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[1698799258105, 1698800293091, 1698802939664, ..."
1,u_1057881,s_33868982,[v_665861],[2],[1698798231452]
2,u_1036784,s_32474802,[v_2594840],[2],[1698799954023]
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[1698800300793, 1698801424774, 1698800040360, ..."
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[1698801260828, 1698800048636, 1698801083807, ..."
...,...,...,...,...,...
95,u_82960,s_10325078,"[v_2278711, v_1035188]","[2, 2]","[1698856864014, 1698856417946]"
96,u_947791,s_30932649,"[v_1190150, v_833279, v_980848, v_256796, v_33...","[2, 2, 2, 2, 2, 2, 2]","[1698856625950, 1698856335812, 1698857165399, ..."
97,u_763919,s_20134506,"[v_1106593, v_626891, v_821107, v_2448709, v_2...","[1, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 2, 2]","[1698856949115, 1698856733568, 1698856497210, ..."
98,u_953016,s_33212705,"[v_2331325, v_1067179, v_500282, v_1925091, v_...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, ...","[1698857695812, 1698857014631, 1698857674186, ..."


In [38]:
def cleaning_vacancies(row):
    ans = set(row) & vacancies
    if bool(ans):
        return None
    else:
        return row
        

In [27]:
df_train

,user_id,session_id,vacancy_id,action_type,action_dt
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[1698799258105, 1698800293091, 1698802939664, ..."
1,u_1057881,s_33868982,[v_665861],[2],[1698798231452]
2,u_1036784,s_32474802,[v_2594840],[2],[1698799954023]
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[1698800300793, 1698801424774, 1698800040360, ..."
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[1698801260828, 1698800048636, 1698801083807, ..."
...,...,...,...,...,...
3463059,u_202578,s_8306993,[v_1499243],[2],[1699909301813]
3463060,u_895531,s_7293998,[v_2041381],[2],[1699984282169]
3463061,u_317562,s_9866576,[v_503192],[2],[1699992234988]
3463062,u_225581,s_19724962,[v_2526106],[2],[1699995171985]


In [39]:
df_train['clear_vacancy_id'] = df_train['vacancy_id'].apply(cleaning_vacancies)

/var/folders/gz/46_zg8xd6qqgn1sxylfq7g8m0000gn/T/ipykernel_3657/42391944.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train_mini['clear_vacancy_id'] = df_train_mini['vacancy_id'].apply(cleaning_vacancies)


In [41]:
df_train_mini.head()

,user_id,session_id,vacancy_id,action_type,action_dt,clear_vacancy_id
0,u_332060,s_28301374,"[v_2571684, v_488179, v_2389179, v_1393783, v_...","[2, 2, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 2, ...","[1698799258105, 1698800293091, 1698802939664, ...",None
1,u_1057881,s_33868982,[v_665861],[2],[1698798231452],None
2,u_1036784,s_32474802,[v_2594840],[2],[1698799954023],[v_2594840]
3,u_786220,s_14060785,"[v_1473781, v_1622905, v_1621959, v_2289180, v...","[1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 3, 2, ...","[1698800300793, 1698801424774, 1698800040360, ...","[v_1473781, v_1622905, v_1621959, v_2289180, v..."
4,u_639152,s_23205986,"[v_695738, v_22433, v_1590524, v_502496, v_200...","[2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 2, ...","[1698801260828, 1698800048636, 1698801083807, ...",None


In [36]:
'v_1393783' in vacancies

True

In [40]:
df_train.dropna(inplace= True)

0      True
1      True
2     False
3     False
4      True
      ...  
95    False
96     True
97     True
98     True
99    False
Name: clear_vacancy_id, Length: 100, dtype: bool

In [ ]:
df_train.to_csv(​ "train.csv"​) #укажи путь куда сохранять 